Importando as bibliotecas e abrindo o vídeo

In [ ]:
import moviepy.editor as mpe
import numpy as np
import vfx
video = mpe.VideoFileClip('video.mp4')

Função para inverter o eixo vertical a cada 20 segundos

In [ ]:

duracao = video.duration

segmentos = []
inicio = 0

while inicio < duracao:
    fim = min(inicio + 20, duracao)
    segmento = video.subclip(inicio, fim)
    
    if (inicio // 20) % 2 == 1:
        segmento = segmento.fx(vfx.mirror_y)
    
    segmentos.append(segmento)
    inicio = fim

video_final = mpe.concatenate_videoclips(segmentos)

video_final.write_videofile("video_invertido.mp4", codec="libx264", audio_codec="aac")

Ajustando o audio para ir diminuindo a cada 30 segundos e ficar sem audio nos últimos 10 segundos

In [ ]:
def volume_function(t, audio_shape):

    mask = t >= duracao_total - 10
    intervals = t // 30
    factors = 1 - 0.1 * intervals
    factors[mask] = 0.0
    return np.tile(np.maximum(factors, 0.0), (audio_shape[1], 1)).T

def aplicar_volume(get_audio_frame, t):
    
    audio_frame = get_audio_frame(t)
    fator = volume_function(t, audio_frame.shape)
    if audio_frame.dtype != fator.dtype:
        fator = fator.astype(audio_frame.dtype)
    return audio_frame * fator

video_invertido_com_audio = mpe.VideoFileClip("video_invertido.mp4")
duracao_total = video_invertido_com_audio.duration

audio_reduzido = video_invertido_com_audio.audio.fl(aplicar_volume)

video_final = video_invertido_com_audio.set_audio(audio_reduzido)

# Salvar o vídeo final
video_final.write_videofile("video_final.mp4", codec="libx264")

Por fim, pegando o tempo de 60 a 80 segundos e passando para o final do video


In [ ]:
video1 = video_final.subclip(0,60)
video2 = video_final.subclip(60,80)
video3 = video_final.subclip(80)
videoCortado = mpe.concatenate_videoclips([video1,video2,video3])

videoCortado.write_videofile('video_cortado.mp4')